In [1]:
import pandas as pd

In [167]:
df = pd.read_csv("./생활이동_행정동_202212/생활이동_행정동_2022.12_01시.csv", encoding="cp949")

### 면허 없는 나이대 0, 5, 10, 15 제거

In [168]:
for i in [0, 5, 10, 15]:
    idx = df[df["나이"] == i].index
    df.drop(idx, inplace=True)

### 모든 요일을 따로보고 성별, 이동유형, 이동시간 고려 X

In [169]:
df.drop(["대상연월", "나이", "성별", "이동유형", "평균 이동 시간(분)"], axis=1, inplace=True)

In [170]:
df.loc[df["이동인구(합)"] == "*", "이동인구(합)"] = 2

In [171]:
df["이동인구(합)"] = df["이동인구(합)"].apply(lambda x: float(x))

### 행정동 코드

In [172]:
id = pd.read_excel("./행정동코드_매핑정보_20200325.xlsx")

In [173]:
id = id[id["시군구명"] == "강서구"]

In [174]:
id.drop(["행자부행정동코드"], axis=1, inplace=True)

In [175]:
id

,통계청행정동코드,시도명,시군구명,행정동명
241,1116051,서울,강서구,염창동
242,1116052,서울,강서구,등촌1동
243,1116053,서울,강서구,등촌2동
244,1116054,서울,강서구,등촌3동
245,1116055,서울,강서구,화곡본동
246,1116057,서울,강서구,화곡2동
247,1116058,서울,강서구,화곡3동
248,1116059,서울,강서구,화곡4동
249,1116061,서울,강서구,화곡6동
250,1116063,서울,강서구,화곡8동


In [176]:
codes = list(id["통계청행정동코드"])

### 출발 & 도착이 같은 경우

In [177]:
df.head()

,요일,도착시간,출발 행정동 코드,도착 행정동 코드,이동인구(합)
0,일,1,1101053,1101053,2.00
1,일,1,1101053,1101053,2.00
2,일,1,1101053,1101053,2.00
3,일,1,1101053,1101053,2.00
4,일,1,1101053,1101053,6.05


In [178]:
tmp = df[df["출발 행정동 코드"].map(lambda x: x in codes)]

In [190]:
tmp

,요일,도착시간,출발 행정동 코드,도착 행정동 코드,이동인구(합)
111721,일,1,1116051,1102052,3.85
111722,일,1,1116051,1102073,3.21
111723,일,1,1116051,1103052,2.00
111724,일,1,1116051,1103074,3.24
111725,일,1,1116051,1104072,6.05
...,...,...,...,...,...
1465988,토,1,1116074,34000,2.00
1465989,토,1,1116074,34000,3.02
1465991,토,1,1116074,36000,2.00
1465993,토,1,1116074,37000,2.00


In [180]:
local = tmp[tmp["출발 행정동 코드"] == tmp["도착 행정동 코드"]]

In [181]:
g_local = local.groupby(["요일", "도착 행정동 코드"]).sum().reset_index()

In [182]:
local_ = pd.merge(left=g_local, right=id, how="inner", left_on="도착 행정동 코드", right_on="통계청행정동코드").drop(["도착 행정동 코드", "출발 행정동 코드", "통계청행정동코드", "시도명", "시군구명"], axis=1)

In [183]:
local = pd.DataFrame()

In [184]:
local = pd.concat([local, local_])

### 출발과 도착이 다른 경우

In [187]:
outflux = tmp[tmp["출발 행정동 코드"] != tmp["도착 행정동 코드"]]

In [192]:
outflux.groupby(["요일", "도착시간", "출발 행정동 코드"]).sum().reset_index()

,요일,도착시간,출발 행정동 코드,도착 행정동 코드,이동인구(합)
0,금,1,1116051,461720641,1079.19
1,금,1,1116052,711291169,1522.48
2,금,1,1116053,158693195,350.90
3,금,1,1116054,1232847012,2607.23
4,금,1,1116055,499476990,1124.74
...,...,...,...,...,...
135,화,1,1116070,388274491,924.31
136,화,1,1116071,599106537,1250.43
137,화,1,1116072,169903972,402.58
138,화,1,1116073,758211462,1642.67


In [111]:
tmp = df[df["도착 행정동 코드"].map(lambda x: x in codes)]

In [112]:
influx = df[df["출발 행정동 코드"] != df["도착 행정동 코드"]]

In [115]:
influx

,요일,도착시간,출발 행정동 코드,도착 행정동 코드,이동인구(합)
35,일,0,1101053,1101055,2.00
36,일,0,1101053,1101055,2.00
37,일,0,1101053,1101055,2.00
38,일,0,1101053,1101055,3.21
39,일,0,1101053,1101055,2.00
...,...,...,...,...,...
2813509,토,0,39000,1125074,3.30
2813511,토,0,39000,1125074,2.00
2813512,토,0,39000,1125074,2.00
2813513,토,0,39000,1125074,2.00


In [128]:
g_influx = influx.groupby(["요일", "도착 행정동 코드"]).sum().reset_index()

In [133]:
influx_ = pd.merge(left=g_influx, right=id, how="inner", left_on="도착 행정동 코드", right_on="통계청행정동코드").drop(["도착 행정동 코드", "출발 행정동 코드", "통계청행정동코드", "시도명", "시군구명"], axis=1)

In [134]:
influx_

,요일,도착시간,이동인구(합),행정동명
0,금,0,3203.13,염창동
1,목,0,2974.56,염창동
2,수,0,2265.91,염창동
3,월,0,1288.68,염창동
4,일,0,2452.72,염창동
...,...,...,...,...
135,수,0,2163.50,우장산동
136,월,0,1375.07,우장산동
137,일,0,2198.83,우장산동
138,토,0,3417.18,우장산동


# 주차면수 대비 불법주정차

In [5]:
df = pd.read_csv("./illegalparking.csv", encoding="cp949")

In [23]:
latest1 = df[df["단속년도"] == 2020]

In [24]:
parking_lot = pd.read_csv("./주차장(동별)(2016년+이후)_20230318155831.csv")

In [25]:
latest2 = parking_lot[parking_lot["동별(2)"] == "강서구"][["동별(3)", "2022.1"]]

In [26]:
latest1

,단속년도,시군구명,행정동명,과태료명,단속건수
100,2020,강서구,가양1동,주정차위반과태료,16889
101,2020,강서구,가양2동,주정차위반과태료,13315
102,2020,강서구,가양3동,주정차위반과태료,1801
103,2020,강서구,공항동,주정차위반과태료,41748
104,2020,강서구,등촌1동,주정차위반과태료,6341
105,2020,강서구,등촌2동,주정차위반과태료,1888
106,2020,강서구,등촌3동,주정차위반과태료,2824
107,2020,강서구,발산1동,주정차위반과태료,20476
108,2020,강서구,방화1동,주정차위반과태료,6578
109,2020,강서구,방화2동,주정차위반과태료,7404


In [31]:
latest2.columns = ["행정동명", "주차면수"]

In [32]:
latest2

,행정동명,주차면수
243,염창동,20668
244,등촌1동,13600
245,등촌2동,9611
246,등촌3동,17670
247,화곡본동,12732
248,화곡2동,6412
249,화곡3동,9879
250,화곡4동,8090
251,화곡6동,14487
252,화곡8동,8322


In [38]:
latest

,단속년도,시군구명,행정동명,과태료명,단속건수
100,2020,강서구,가양1동,주정차위반과태료,16889
101,2020,강서구,가양2동,주정차위반과태료,13315
102,2020,강서구,가양3동,주정차위반과태료,1801
103,2020,강서구,공항동,주정차위반과태료,41748
104,2020,강서구,등촌1동,주정차위반과태료,6341
105,2020,강서구,등촌2동,주정차위반과태료,1888
106,2020,강서구,등촌3동,주정차위반과태료,2824
107,2020,강서구,발산1동,주정차위반과태료,20476
108,2020,강서구,방화1동,주정차위반과태료,6578
109,2020,강서구,방화2동,주정차위반과태료,7404


In [39]:
latest = pd.merge(left=latest1, right=latest2, left_on="행정동명", right_on="행정동명", how="inner")[["행정동명", "주차면수"]]

In [43]:
latest = pd.merge(left=latest1, right=latest2, left_on="행정동명", right_on="행정동명", how="inner").iloc[:, [2, 4, 5]]

In [50]:
latest.iloc[:, 1] = latest.iloc[:, 1].apply(lambda x: int(x))

In [52]:
latest.iloc[:, 2] = latest.iloc[:, 2].apply(lambda x: int(x))

In [53]:
latest["면수대비건수"] = latest.iloc[:, 1] / latest["주차면수"]

In [54]:
latest

,행정동명,단속건수,주차면수,면수대비건수
0,가양1동,16889,49535,0.340951
1,가양2동,13315,5314,2.505645
2,가양3동,1801,5567,0.323514
3,공항동,41748,25818,1.617011
4,등촌1동,6341,13600,0.466250
5,등촌2동,1888,9611,0.196442
6,등촌3동,2824,17670,0.159819
7,발산1동,20476,29360,0.697411
8,방화1동,6578,14206,0.463044
9,방화2동,7404,15923,0.464988
